In [75]:
from prepocessing import DataProcessor

In [76]:
Processor = DataProcessor()
X_train, X_test, y_train, y_test = Processor.get_datas()

In [77]:
from Modello import model
from Training import training

In [ ]:
model_1=model(num_k=[32,64,128], k_size=[3,3,3], p_size=[2,2,2], num_n=256)
training_1=training(model=model_1.model,X_train=X_train,y_train=y_train, ep=5, b_size=32, val_split=0.1)

In [ ]:
training_1.grafico_accuracy()
training_1.grafico_loss()

In [68]:
from Testing import testing

In [ ]:
testing_1=testing(model=training_1.trained_model, X_test=X_test, y_test=y_test)
print(testing_1.test_loss, testing_1.test_accuracy)